# About

test to see if we can get better than 20% accuracy based on the various time features created by SFCCTransformer and one hot encoding

In [39]:
import numpy as np
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.neighbors import KNeighborsClassifier

from pandas.api.types import CategoricalDtype
from plotnine import *

%reload_ext autoreload
%autoreload 2
from src.features.build_features import DataFrameSelector, SFCCTransformer, print_summary

import inspect

In [40]:
# loads data
train_pd = pd.read_csv("../data/raw/train.csv.zip", compression="zip")
test_pd = pd.read_csv("../data/raw/test.csv.zip", compression="zip")

In [41]:
# shuffles, then splits into train and dev sets
shuffled = train_pd.sample(frac=1, random_state = 0)
split = round(train_pd.shape[0] * 0.8)
train_data = shuffled[:split]
dev_data = shuffled[split+1:]

print(split)
print(train_data.shape)
print(dev_data.shape)

# gets the train and dev labels
train_labels = train_data["Category"]
dev_labels = dev_data["Category"]

print(train_labels.shape)
print(dev_labels.shape)

702439
(702439, 9)
(175609, 9)
(702439,)
(175609,)


In [42]:
# extracts some features from the raw train and test data frames
pipe = Pipeline([
    ("transformer", SFCCTransformer()),
    ("selector", DataFrameSelector(attribute_names = ["hour_of_day_sin", "hour_of_day_cos"])),
])

train_prep = pipe.transform(train_data)
dev_prep = pipe.transform(dev_data)
test_prep = pipe.transform(test_pd)


In [43]:
# some eda stuff
print(train_prep.shape)
print(dev_prep.shape)
print(test_prep.shape)

print(train_prep[0:3])
print(dev_prep[0:3])
print(test_prep[0:3])


(702439, 2)
(175609, 2)
(884262, 2)
[[-0.966 -0.259]
 [-1.    -0.   ]
 [-0.866 -0.5  ]]
[[ 0.259  0.966]
 [-0.966 -0.259]
 [-0.259  0.966]]
[[-0.259  0.966]
 [-0.259  0.966]
 [-0.259  0.966]]


In [ ]:
# create classifier pipeline
clf_pipeline = Pipeline(steps = [
    ('knn', KNeighborsClassifier())
])

# see 
# https://scikit-learn.org/stable/tutorial/statistical_inference/putting_together.html
# https://stackoverflow.com/questions/42266737/parallel-pipeline-to-get-best-model-using-gridsearch
param_grid = {
    'knn__n_neighbors': [3, 5, 9, 17, 23]
}
search = GridSearchCV(clf_pipeline, param_grid, iid = False, cv = 5, return_train_score = False)

_ = search.fit(train_prepared, train_labels)
print("Best parameter (CV score=%0.3f):" % search.best_score_)
print(search.best_params_)

# Now evaluate all steps on test set
score = search.score(dev_prepared, dev_labels)
print(score)